# MODELLI IR - Spazio vettoriale

Librerie -->nltk e scikit-learn

## CountVectorizer
**CouterVectorizer** classe per convertire il testo in una matrice di token. Usa la tf per la pesatura dei token e 
produce un modello con matrici sparse di tipo Numpy. Usa il modello **bag of word**.
Di default fa già 
- una tokenizzazione 
- la creazione del vocabolario
- costruzione della matrice

Il costruttore se usato senza argomenti usa le sue impostazioni di default che pero è possibile modificare passando funzioni custom.
Fa test pre-processing ma non rimuove le stopword.

In [117]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


vectorizer=CountVectorizer()

#documenti da rappresentare nello spazio vettoriale
#ogni riga è un documento
corpus=["Racing games",
        "This document describes racing cars",
        "This document is about video games in general",
        "This is a nice racing video game"]

Per creare il modello usiamo:

In [118]:
mod_vect=vectorizer.fit_transform(corpus)

df = pd.DataFrame(mod_vect.toarray(), columns=vectorizer.get_feature_names_out())
print(df)


   about  cars  describes  document  game  games  general  in  is  nice  \
0      0     0          0         0     0      1        0   0   0     0   
1      0     1          1         1     0      0        0   0   0     0   
2      1     0          0         1     0      1        1   1   1     0   
3      0     0          0         0     1      0        0   0   1     1   

   racing  this  video  
0       1     0      0  
1       1     1      0  
2       0     1      1  
3       1     1      1  


che serve per creare il mdello dai documenti, quindi impara il vocabolario e lo trasforma in una matrice termini-documenti.

**Valori di ritorno** --> matrice sparsa Scipy, dove le riche sono i documenti e le colonne gli index termo per rappresentare i documenti nello spazio vettoriale. Nelle intersezioni si ha la tf, cioè la frequenza con cui una parola appare in quel documento.

**Parametri** --> lista dei documenti, iterabile

Per apprendere dai dati solo le statistiche per creare il modello senza apprendere il modello, quindi senza effettuare una trasformazione dei dati trasformandoli in una forma matriciale, ma calcolare solo il vocabolario usiamo

In [119]:
stats=vectorizer.fit(corpus)
print(stats) 
print(stats.get_feature_names_out())


CountVectorizer()
['about' 'cars' 'describes' 'document' 'game' 'games' 'general' 'in' 'is'
 'nice' 'racing' 'this' 'video']


**Valori di ritorno** --> un oggetto CountVectorizer, ma con le informazioni per creare il modello

**Parametri** --> lista dei documenti, iterabile

Poi usando questo nuovo CountVectorizer possiamo creare il modello usando

In [120]:
mtd=stats.transform(corpus)
df = pd.DataFrame(mtd.toarray(), columns=vectorizer.get_feature_names_out())
print(df)

   about  cars  describes  document  game  games  general  in  is  nice  \
0      0     0          0         0     0      1        0   0   0     0   
1      0     1          1         1     0      0        0   0   0     0   
2      1     0          0         1     0      1        1   1   1     0   
3      0     0          0         0     1      0        0   0   1     1   

   racing  this  video  
0       1     0      0  
1       1     1      0  
2       0     1      1  
3       1     1      1  


questa funzione è usata anche per poter trasformare i nuovi dati da inserire nel modello. In questo caso il modello non viene ricreato, ma il nuovo documento che viene inserito viene trasposto nello spazio vettoriale già esistente considerando i termini presenti nello spazio. termini che non sono mai stati visti non vengono inseriti, ma scartati.

## Cosine Similarity
è una funzione che calcola il coseno fra due vettori. In questo caso il coseno rappresenta la similarità fra due documenti.
1. se passiamo alla funzione solo la matrice del modello calcola la cos_sim fra i documenti usati per addrestrare il modello;
2. se passiamo alla funzione la matrice del modello e la query, la cos_sim è calcolata fra la query e ogni singolo documeto. In questo caso la query deve essere prima inserita nello spazio vettoriale con la  *fit_trasform()*.

In [121]:
# Calcolare la similarità coseno tra tutti i documenti
cos = cosine_similarity(mod_vect)
print("Similarità Coseno tra i documenti:")
print(cos)
print()

# Aggiungere una query al modello
query = vectorizer.transform(["racing game"])

# Calcolare la similarità coseno tra la query e i documenti
cos = cosine_similarity(query, mod_vect)
print("Similarità Coseno tra la query e i documenti:")
print(cos)

Similarità Coseno tra i documenti:
[[1.         0.31622777 0.25       0.28867513]
 [0.31622777 1.         0.31622777 0.36514837]
 [0.25       0.31622777 1.         0.4330127 ]
 [0.28867513 0.36514837 0.4330127  1.        ]]

Similarità Coseno tra la query e i documenti:
[[0.5        0.31622777 0.         0.57735027]]


# MODELLI IR - Tf-idf

## TfidfVectorizer
TfidfVectorizer è una classe che usa come metrica la tf-idf, funziona come il caso precedente.

# MODELLI IR - Inverted Index

Librerie --> gensim

Si potrebbe fare anche con scikit-learn usando **HashingVectorizer** ma usa come metrica la tf. Con gensim usando **Corpora.Dictionary** mappiamo ogni parola del testo ad un ID univoco, quindi usando un modello BoW

In [122]:
from gensim import corpora
from gensim import models
from gensim import similarities
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import snowball
import re

def my_tokenizer(text):
    """tokenization function"""
    sw=stopwords.words('english')
    stemmer=snowball.SnowballStemmer(language="english")
    tokens=word_tokenize(text)
    pruned=[stemmer.stem(t.lower()) for t in tokens \
            if re.search(r"^\w",t) and not t.lower() in sw]
    return pruned

documents=["This document describes racing cars",
        "This document is about video games in general",
        "This is a nice racing video game"]

texts=[my_tokenizer(d) for d in documents]

dictionary = corpora.Dictionary(texts)
print(dictionary)

# doc2bow per convertire i docs in BoW 
bow_corpus=[dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(bow_corpus)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## Similarities.SpaseMatrixSimilarity
La funzione **`SparseMatrixSimilarity`** è una classe di Gensim utilizzata per calcolare la similarità tra una query e un insieme di documenti, rappresentati come matrici sparse (in formato **TF-IDF** o **Bag-of-Words**).

La funzione **`SparseMatrixSimilarity`** accetta i seguenti parametri:

### 1. **`corpus`** 
- **Descrizione**: La matrice sparsa che rappresenta il corpus di documenti.
- **Tipo**: Una matrice sparsa, generalmente una rappresentazione **TF-IDF** o **Bag-of-Words**.
- **Esempio**: `tfidf[bow_corpus]`
- **Funzione**: Il corpus è una matrice che contiene la rappresentazione numerica di tutti i documenti, dove le righe sono i documenti e le colonne sono i termini del vocabolario. Ogni valore nella matrice rappresenta l'importanza di un termine in un dato documento.

### 2. **`num_features`** 
- **Descrizione**: Il numero di **caratteristiche** (o dimensioni) nel vocabolario.
- **Tipo**: Un intero che rappresenta la lunghezza del vocabolario.
- **Esempio**: `len(dictionary)` (dove `dictionary` è il vocabolario che contiene tutte le parole uniche nei documenti).
- **Funzione**: Specifica la dimensione del vocabolario, ossia il numero di parole uniche presenti nel tuo corpus. Questo parametro è necessario per indicare la dimensione della matrice sparsa.

### 3. **`num_best`** 
- **Descrizione**: Il numero di documenti più simili da restituire.
- **Tipo**: Un intero (predefinito è 10).
- **Esempio**: `num_best=5`
- **Funzione**: Limita il numero di risultati restituiti, mostrando solo i documenti più simili alla query. Se non specificato, restituirà tutti i documenti con le loro similarità coseno rispetto alla query.

### 4. **`threshold`** 
- **Descrizione**: Una soglia di similarità minima. Se la similarità coseno tra un documento e la query è inferiore a questa soglia, il documento verrà escluso dai risultati.
- **Tipo**: Un valore float tra 0 e 1 (predefinito è 0.0).
- **Esempio**: `threshold=0.5`
- **Funzione**: Filtra i documenti con una similarità inferiore alla soglia specificata. Se impostato a 0.5, solo i documenti con una similarità maggiore o uguale a 0.5 saranno restituiti.

---

La funzione **`SparseMatrixSimilarity`** restituisce un oggetto che può essere utilizzato per calcolare la similarità coseno tra una query e il corpus di documenti.

### Ritorno:
- **Tipo**: Un oggetto di tipo **`SparseMatrixSimilarity`**.
- **Funzione**: L'oggetto restituito è utilizzato per calcolare la similarità tra un dato documento e il corpus di documenti. Quando si fornisce una query a questo oggetto, restituirà un array di similarità coseno tra la query e ogni documento nel corpus.


In [ ]:

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus],len(dictionary))
print(list(index))

#tokenizzazione della query
query_document = my_tokenizer("racing games")
query_bow = dictionary.doc2bow(query_document)

sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))
